In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
column_names = ["pmid", "entity1_type", "entity2_type", "entity1_id", "entity2_id",
                "random_bool", "random_integer", "text", "label", "novel"]

train_df = pd.read_csv("processed2/train.tsv", sep="\t", header=None, names=column_names)
test_df = pd.read_csv("processed2/test.tsv", sep="\t", header=None, names=column_names)

train_df.drop(columns=["pmid", "entity1_type", "entity2_type", "entity1_id", "entity2_id",
                "random_bool", "random_integer", "novel"], inplace=True)
test_df.drop(columns=["pmid", "entity1_type", "entity2_type", "entity1_id", "entity2_id",
                "random_bool", "random_integer", "novel"], inplace=True)

train_df.fillna({"label": "None"}, inplace=True)
test_df.fillna({"label": "None"}, inplace=True)

label_mapping = {'Negative_Correlation': 0, 'Association': 1, 'Drug_Interaction': 2, 'Cotreatment': 3, 'None': 4, 'Conversion': 5, 'Positive_Correlation': 6, 'Comparison': 7, 'Bind': 8}

train_df['label'] = train_df['label'].map(label_mapping)
test_df['label'] = test_df['label'].map(label_mapping)

# Display first few rows to inspect the structure
print(train_df.head())

                                                text  label
0  @GeneOrGeneProductSrc$ Hepatocyte nuclear fact...      4
1  Hepatocyte nuclear factor-6 : associations bet...      4
2  Hepatocyte nuclear factor-6 : associations bet...      4
3  @GeneOrGeneProductSrc$ Hepatocyte nuclear fact...      4
4  Hepatocyte nuclear factor-6 : associations bet...      4


In [ ]:
train = Dataset.from_pandas(train_df)
test = Dataset.from_pandas(test_df)

In [ ]:
train[0]

{'text': '@GeneOrGeneProductSrc$ Hepatocyte nuclear factor-6 @/GeneOrGeneProductSrc$ : associations between genetic variability and type II diabetes and between genetic variability and estimates of insulin secretion . The transcription factor @GeneOrGeneProductSrc$ hepatocyte nuclear factor (HNF)-6 @/GeneOrGeneProductSrc$ is an upstream regulator of several genes involved in the pathogenesis of maturity-onset diabetes of the young . We therefore tested the hypothesis that variability in the @GeneOrGeneProductSrc$ HNF-6 @/GeneOrGeneProductSrc$ gene is associated with subsets of Type II ( non-insulin-dependent ) diabetes mellitus and estimates of insulin secretion in @ChemicalEntityTgt$ glucose @/ChemicalEntityTgt$ tolerant subjects . We cloned the coding region as well as the intron-exon boundaries of the @GeneOrGeneProductSrc$ HNF-6 @/GeneOrGeneProductSrc$ gene . W e then examined them on genomic DNA in six MODY probands without mutations in the MODY1 , MODY3 and MODY4 genes and in 54 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(dataset):
    return tokenizer(dataset["text"], truncation=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/30830 [00:00<?, ? examples/s]

Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

In [ ]:
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

In [ ]:
# tokenized_test[0]

{'text': 'A novel SCN5A mutation manifests as a malignant form of long QT syndrome with perinatal onset of tachycardia/bradycardia . OBJECTIVE : Congenital long QT syndrome ( LQTS ) with in utero onset of the rhythm disturbances is associated with a poor prognosis . In this study we investigated a newborn patient with fetal bradycardia , 2:1 @DiseaseOrPhenotypicFeatureSrc$ atrioventricular block @/DiseaseOrPhenotypicFeatureSrc$ and ventricular tachycardia soon after birth . METHODS : Mutational analysis and DNA sequencing were conducted in a newborn . The 2:1 @DiseaseOrPhenotypicFeatureSrc$ atrioventricular block @/DiseaseOrPhenotypicFeatureSrc$ improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine , which also controlled the ventricular tachycardia . RESULTS : A novel , spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel , with a @SequenceVariantTgt$ G-->A substitution

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=9)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0166, -0.0665, -0.0163,  ..., -0.0200, -0.0513, -0.0263],
         [-0.0132, -0.0673, -0.0160,  ..., -0.0226, -0.0553, -0.0260],
         [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0595, -0.0232],
         ...,
         [-0.0231, -0.0587, -0.0105,  ..., -0.0194, -0.0261, -0.0212],
         [-0.0490, -0.0561, -0.0046,  ..., -0.0106, -0.0180, -0.0219],
         [-0.0065, -0.0914, -0.0025,  ..., -0.0150, -0.0504,  0.0459]],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[ 0.0187, -0.0267, -0.0353,  ...,  0.0010, -0.0002,  0.0146],
         [ 0.0073,  0.0022, -0.0148,  ...,  0.0304,  0.0307, -0.0038],
         [-0.0145, -0.0013, -0.0123,  ...,  0.0123,  0.0206, -0.0066],
         ...,
         [ 0.0195,  0.0028, -0.0110,  ...,  0.0033, -0.0006, -0.0323],
         [ 0.0230, -0.0017,  0.0171,  ..., -0.0053, -0.0105, -0.0315],
         [ 0.0022, -0.0225,  0.0078,  ...,  0.0150,  0.0261, -0.0769]],
        device='cuda:0', r

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,

)

trainer.train()

<ipython-input-12-638e0b2750a6>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.526700
1000,0.430000
1500,0.373900
2000,0.353500
2500,0.340600
3000,0.329200
3500,0.306500
4000,0.301200
4500,0.263500
5000,0.277600


TrainOutput(global_step=9635, training_loss=0.2817254581659274, metrics={'train_runtime': 7626.7514, 'train_samples_per_second': 20.212, 'train_steps_per_second': 1.263, 'total_flos': 2.04223986160128e+16, 'train_loss': 0.2817254581659274, 'epoch': 5.0})

In [ ]:
# tokenized_test[0]
trainer.evaluate(eval_dataset=tokenized_test)

{'eval_loss': 0.44794994592666626,
 'eval_runtime': 143.9885,
 'eval_samples_per_second': 69.742,
 'eval_steps_per_second': 4.361,
 'epoch': 5.0}

In [ ]:
trainer.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import torch

In [ ]:
tokenized_test[0]

{'text': 'A novel SCN5A mutation manifests as a malignant form of long QT syndrome with perinatal onset of tachycardia/bradycardia . OBJECTIVE : Congenital long QT syndrome ( LQTS ) with in utero onset of the rhythm disturbances is associated with a poor prognosis . In this study we investigated a newborn patient with fetal bradycardia , 2:1 @DiseaseOrPhenotypicFeatureSrc$ atrioventricular block @/DiseaseOrPhenotypicFeatureSrc$ and ventricular tachycardia soon after birth . METHODS : Mutational analysis and DNA sequencing were conducted in a newborn . The 2:1 @DiseaseOrPhenotypicFeatureSrc$ atrioventricular block @/DiseaseOrPhenotypicFeatureSrc$ improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine , which also controlled the ventricular tachycardia . RESULTS : A novel , spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel , with a @SequenceVariantTgt$ G-->A substitution

In [ ]:
tokenized_test[0]

{'text': 'A novel SCN5A mutation manifests as a malignant form of long QT syndrome with perinatal onset of tachycardia/bradycardia . OBJECTIVE : Congenital long QT syndrome ( LQTS ) with in utero onset of the rhythm disturbances is associated with a poor prognosis . In this study we investigated a newborn patient with fetal bradycardia , 2:1 @DiseaseOrPhenotypicFeatureSrc$ atrioventricular block @/DiseaseOrPhenotypicFeatureSrc$ and ventricular tachycardia soon after birth . METHODS : Mutational analysis and DNA sequencing were conducted in a newborn . The 2:1 @DiseaseOrPhenotypicFeatureSrc$ atrioventricular block @/DiseaseOrPhenotypicFeatureSrc$ improved to 1:1 conduction only after intravenous lidocaine infusion or a high dose of mexiletine , which also controlled the ventricular tachycardia . RESULTS : A novel , spontaneous LQTS-3 mutation was identified in the transmembrane segment 6 of domain IV of the Na(v)1.5 cardiac sodium channel , with a @SequenceVariantTgt$ G-->A substitution

In [ ]:
tokenized_test

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 10042
})

In [ ]:
tokenized_test_removed = tokenized_test.remove_columns(['text'])

In [ ]:
test_dataloader = DataLoader(tokenized_test_removed, batch_size=16, collate_fn=data_collator)

In [ ]:
next(iter(test_dataloader))

{'input_ids': tensor([[  101,  1037,  3117,  ...,     0,     0,     0],
        [  101,  1037,  3117,  ...,  8581, 23892,   102],
        [  101,  1037,  3117,  ...,     0,     0,     0],
        ...,
        [  101,  1037,  3117,  ...,  1012,   102,     0],
        [  101,  1037,  3117,  ...,     0,     0,     0],
        [  101,  1037,  3117,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([4, 4, 4, 4, 4, 4, 4, 4, 1, 0, 4, 4, 0, 4, 4, 4])}

In [ ]:
test_dataloader = DataLoader(tokenized_test_removed, batch_size=16, collate_fn=data_collator)

# Lists to store predictions and true labels
predictions = []
true_labels = []

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for batch in test_dataloader:
        # Move the batch to the appropriate device (CPU or GPU)
        inputs = {key: val.to('cuda') for key, val in batch.items() if key != 'label'}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].numpy())

# Generate classification report


AttributeError: 'Value' object has no attribute 'feature'

In [ ]:
report = classification_report(true_labels, predictions, target_names=label_mapping.keys())
print(report)

                      precision    recall  f1-score   support

Negative_Correlation       0.28      0.23      0.25       171
         Association       0.36      0.30      0.33       635
    Drug_Interaction       0.00      0.00      0.00         2
         Cotreatment       0.29      0.14      0.19        14
                None       0.94      0.96      0.95      8879
          Conversion       0.00      0.00      0.00         1
Positive_Correlation       0.39      0.36      0.37       325
          Comparison       0.00      0.00      0.00         6
                Bind       0.00      0.00      0.00         9

            accuracy                           0.88     10042
           macro avg       0.25      0.22      0.23     10042
        weighted avg       0.87      0.88      0.88     10042



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
model_2 = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=9)

for param in model_2.base_model.parameters():
    param.requires_grad = False

training_args = TrainingArguments(
    output_dir="./results-finetune",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="none"
)

trainer = Trainer(
    model=model_2,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,

)

trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-56-e2b664c1587f>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.745900
1000,0.531000
1500,0.507700
2000,0.498000
2500,0.503000
3000,0.479900
3500,0.479600
4000,0.490200
4500,0.466600
5000,0.486000


TrainOutput(global_step=9635, training_loss=0.49671924791004474, metrics={'train_runtime': 2437.2152, 'train_samples_per_second': 63.248, 'train_steps_per_second': 3.953, 'total_flos': 2.04223986160128e+16, 'train_loss': 0.49671924791004474, 'epoch': 5.0})

In [ ]:
test_dataloader = DataLoader(tokenized_test_removed, batch_size=16, collate_fn=data_collator)

# Lists to store predictions and true labels
predictions = []
true_labels = []

model_2.eval()  # Set the model_2 to evaluation mode
with torch.no_grad():
    for batch in test_dataloader:
        # Move the batch to the appropriate device (CPU or GPU)
        inputs = {key: val.to('cuda') for key, val in batch.items() if key != 'label'}
        outputs = model_2(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].numpy())

# Generate classification report
report = classification_report(true_labels, predictions, target_names=label_mapping.keys())
print(report)

                      precision    recall  f1-score   support

Negative_Correlation       0.00      0.00      0.00       171
         Association       0.46      0.03      0.05       635
    Drug_Interaction       0.00      0.00      0.00         2
         Cotreatment       0.00      0.00      0.00        14
                None       0.89      1.00      0.94      8879
          Conversion       0.00      0.00      0.00         1
Positive_Correlation       0.00      0.00      0.00       325
          Comparison       0.00      0.00      0.00         6
                Bind       0.00      0.00      0.00         9

            accuracy                           0.89     10042
           macro avg       0.15      0.11      0.11     10042
        weighted avg       0.81      0.89      0.83     10042



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
report = classification_report(true_labels, predictions, target_names=label_mapping.keys(), digits=4)
print(report)

NameError: name 'classification_report' is not defined